# 1. DATA 

In [3]:
import torch 
from torch import nn 
import numpy as np
import argparse
from Dataset import FairytalesDataset
from torch import nn, optim
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint, train
from models import lstm_model
from test import predict 

In [7]:
DIR_PATH = "data/fairytales.txt"
START_TOKEN = "<s>"
END_TOKEN = "</s>"

parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=0)
parser.add_argument('--batch-size', type=int, default=256)
parser.add_argument('--sequence_length', type=int, default=7)
args, unknown = parser.parse_known_args()

dataset = FairytalesDataset(DIR_PATH, START_TOKEN, END_TOKEN, args)
print(dataset.__getitem__(3))
print('Vocab:', len(dataset.unique_words))

(tensor([ 53,   9,  73,   6,  19, 149,  40]), tensor([  9,  73,   6,  19, 149,  40,  93]))
Vocab: 3028


1 batch = 256 sequences of 7 words
In total, to cover the entire text, we have 146 batches.

# 2. MODEL

Standard lstm model 

In [8]:
model = lstm_model(dataset)
train(dataset, model, args, load_model=True)

-> Loading checkpoint


[]

# 3. TESTING 

In [10]:
text,sent_perplexity = predict(dataset, model, text='there was once', next_words=100)

print('Generated text:')
print(' ')
print(text)
print(' ')
print('Number of sentences:', len(sent_perplexity))
print(' ')
print('Perplexity per sentence:', sent_perplexity)

Generated text:
 
<s> there was once a king son who told his father that he had gone too before directly he had dazzled to make his grain </s> <s> one next fortune so what i wish to enter sure </s> <s> swiftly i will tell him with your majesty and asked that he will let me your answer and she about how he was different but before two asked that she was likely on a best great prince is three one though something the prince </s> <s> i did not carry lucky or three young man i have over the world and this first has left </s>
 
Number of sentences: 4
 
Perplexity per sentence: [1.197037462534054, 1.8803108189588493, 1.92846939064602, 1.9034254316188541]


# 3. METRIC 

## Perplexity 

**Perplexity** refers to the power of a probability distribution to predict, or assign probabilities, to a sample. 

Lower the perplexity value, the better the model. 

A model’s worst-case perplexity is fixed by the language’s vocabulary size.
- If the model is completely dumb(worst possible), perplexity = |v| i.e. size of the vocabulary. 

Perplexity is model dependent

what we want is to compute the perplexity for each word 
Then compute compute the perplexity for each sentence of the text 


Cross-Entropy: how do we use the minimum amount of the memory to store a sequence?

Logits is the vector of raw (non-normalized) predictions that the model generates, which is ordinarily then passed to a normalization function. In this case, logits becomes the input to the softmax function. The softmax function then generates a vector of (normalized) probabilities with one value for each possible word.